In [0]:
from libs.rename_columns import *
from libs.create_table import create_table

In [0]:
path_source = '/Volumes/landing/raw/rfb_cnpj'
catalog = 'projeto_rfb'
schema  = 'bronze'
tables  = ['cnaes', 'empresas', 'estabelecimentos', 'socios', 'motivos', 'municipios', 'naturezas', 'paises', 'qualificacoes', 'simples']

In [0]:
for table in tables:
    # Criando tabela se ela não existir
    create_table(
        spark=spark, 
        catalog=catalog, 
        schema=schema, 
        table=table
    )

    # realizando a leitura dos arquivos e criando um df spark
    df = (spark.read
        .option('delimiter', ';')
        .option('header', False)
        .option('inferSchema', False)
        .option('encoding', 'latin1')
        .csv(f"{path_source}/{table}*")
    )

    table_columns = eval(table)
    # Renomear as colunas
    for old, new in table_columns.items():
        df = df.withColumnRenamed(old, new)

    # Escrevendo em formato delta
    (df.write
        .mode('overwrite')
        .format('delta')
        .option("mergeSchema", "true")
        .saveAsTable(f"{catalog}.{schema}.{table}")
    )

    print(f"{table} populada com sucesso")

# Limpando os arquivos da pasta
list_folders = dbutils.fs.ls(path_source)
for file in list_folders:
    if file.name != 'temp/':
        dbutils.fs.rm(f"{path_source}/{file.name}", True)
